### CE88: Data Science for Smart Cities - 10/9/17

# *Naive Bayes*

Some imports first:

In [1]:
from datascience import *
import numpy as np

### Loading the data

In [2]:
Init_table = Table.read_table('house_vote.csv')
Init_table

### Dividing data into test and train sets

In [4]:
table_train = Table.read_table('house_train.csv')
table_test = Table.read_table('house_test_without_label.csv')

### Finding the prior distribution over class

$p(Y)$

In [5]:
rep_table = table_train.where('V1','republican')
num_rep = rep_table.num_rows
dem_table = table_train.where('V1','democrat')
num_dem = dem_table.num_rows

In [6]:
Y = [(num_rep / table_train.num_rows), (num_dem / table_train.num_rows)]
Y

[0.3834355828220859, 0.6165644171779141]

### Finding the conditional likelihood

$p(X_i \mid Y)$

In [7]:
X_rep = np.zeros(((table_train.num_columns-2),2))
X_dem = np.zeros(((table_train.num_columns-2),2))

for i in range(1,(len(X_rep) + 1)):
    X_rep[(i-1),0] = (sum(rep_table[i+1] == 'y')) / (rep_table.num_rows)
    X_rep[(i-1),1] = (sum(rep_table[i+1] == 'n')) / (rep_table.num_rows)
    X_rep[(i-1),] = X_rep[(i-1),] / sum(X_rep[(i-1),])
    X_dem[(i-1),0] = (sum(dem_table[i+1] == 'y')) / (dem_table.num_rows)
    X_dem[(i-1),1] = (sum(dem_table[i+1] == 'n')) / (dem_table.num_rows)
    X_dem[(i-1),] = X_dem[(i-1),] / sum(X_dem[(i-1),])


### First testing on training data

In [28]:
X_train = np.empty((table_train.num_rows,16))
for i in range(1,table_train.num_rows):
    x = np.asarray(table_train.rows[(i-1)][2:18])
    x[x == 'y'] = 0; x[x == 'n'] = 1
    X_train[(i-1),] = x.astype(np.float)

### Calculating the posterior of class given data for train set

$p(Y \mid X)$

In [11]:
Y_train = np.zeros((len(X_train),2))

for i in range(1,len(Y_train)):
    x = X_train[(i-1),]
    y_rep = 1
    y_dem = 1
    for j in range(1,len(x)):
        if ~np.isnan(x[(j-1)]):
            y_rep = y_rep * X_rep[(j-1),x[(j-1)].astype(np.int)]
            y_dem = y_dem * X_dem[(j-1),x[(j-1)].astype(np.int)]
    Y_train[(i-1),0] =  y_rep * Y[0]
    Y_train[(i-1),1] =  y_dem * Y[1]
    Y_train[(i-1),] = Y_train[(i-1),] / sum(Y_train[(i-1),])

In [20]:
Labels = np.argmax(Y_train,axis = 1)


### Estimating the training accuracy in prediciton

In [18]:
Truth = table_train.columns[1]
Truth[Truth == 'democrat'] = 1
Truth[Truth == 'republican'] = 0
Truth = Truth.astype(np.int)

In [19]:
sum(Labels == Truth) / len(Labels)

0.92024539877300615

### Preparing the test data

In [23]:
X_test = np.empty((table_test.num_rows,16))
for i in range(1,table_test.num_rows):
    x = np.asarray(table_test.rows[(i-1)][1:17])
    x[x == 'y'] = 0; x[x == 'n'] = 1
    X_test[(i-1),] = x.astype(np.float)


### Calculating the posterior of class given data for test set

$p(Y \mid X)$

In [24]:
Y_test = np.zeros((len(X_test),2))

for i in range(1,len(Y_test)):
    x = X_test[(i-1),]
    y_rep = 1
    y_dem = 1
    for j in range(1,len(x)):
        if ~np.isnan(x[(j-1)]):
            y_rep = y_rep * X_rep[(j-1),x[(j-1)].astype(np.int)]
            y_dem = y_dem * X_dem[(j-1),x[(j-1)].astype(np.int)]
    Y_test[(i-1),0] =  y_rep * Y[0]
    Y_test[(i-1),1] =  y_dem * Y[1]
    Y_test[(i-1),] = Y_test[(i-1),] / sum(Y_test[(i-1),])

### Estimating the test accuracy in prediciton

In [25]:
Labels = np.argmax(Y_test,axis = 1)

In [26]:
test = Table.read_table('house_test_with_label.csv')
Truth = test.columns[1]
Truth[Truth == 'democrat'] = 1
Truth[Truth == 'republican'] = 0
Truth = Truth.astype(np.int)

In [27]:
sum(Labels == Truth) / len(Labels)

0.87155963302752293